In [62]:
import nltk, re, pprint

from urllib import request
from bs4 import BeautifulSoup                                                                                   # needed for parsing HTML

import contractions                                                                                             # contractions dictionary
from string import punctuation

import spacy                                                                                                    # used for lemmatization/stemming
#!python -m spacy download en_core_web_sm                # OR in Jupyter download in terminal using spacy download en_core_web_sm

from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
tokenizer = ToktokTokenizer()                                                                                   # stopword removal
from nltk import word_tokenize

import pandas as pd
import numpy as np  

In [63]:
# transcripts from democratic debates for 2020 election

dem_urls = [
    'https://www.presidency.ucsb.edu/documents/democratic-candidates-debate-washington-dc',
    'https://www.presidency.ucsb.edu/documents/democratic-candidates-debate-charleston-south-carolina-0',
    'https://www.presidency.ucsb.edu/documents/democratic-candidates-debate-las-vegas-nevada-0',
    'https://www.presidency.ucsb.edu/documents/democratic-candidates-debate-manchester-new-hampshire-0',
    'https://www.presidency.ucsb.edu/documents/democratic-candidates-debate-des-moines-iowa-0',
    'https://www.presidency.ucsb.edu/documents/democratic-candidates-debate-los-angeles-california',
    'https://www.presidency.ucsb.edu/documents/democratic-candidates-debate-atlanta-georgia',
    'https://www.presidency.ucsb.edu/documents/democratic-candidates-debate-westerville-ohio',
    'https://www.presidency.ucsb.edu/documents/democratic-candidates-debate-houston-texas',
    'https://www.presidency.ucsb.edu/documents/democratic-candidates-debate-detroit-michigan-group-2',
    'https://www.presidency.ucsb.edu/documents/democratic-candidates-debate-detroit-michigan-group-1',
    'https://www.presidency.ucsb.edu/documents/democratic-candidates-debate-miami-florida-group-2',
    'https://www.presidency.ucsb.edu/documents/democratic-candidates-debate-miami-florida-group-1'
]

# transcripts from republican debates for 2016 election

rep_urls = [
    'https://www.presidency.ucsb.edu/documents/republican-candidates-debate-miami-florida',
    'https://www.presidency.ucsb.edu/documents/republican-candidates-debate-detroit-michigan',
    'https://www.presidency.ucsb.edu/documents/republican-candidates-debate-houston-texas',
    'https://www.presidency.ucsb.edu/documents/republican-candidates-debate-greenville-south-carolina',
    'https://www.presidency.ucsb.edu/documents/republican-candidates-debate-manchester-new-hampshire-0',
    'https://www.presidency.ucsb.edu/documents/republican-candidates-debate-des-moines-iowa-0',
    'https://www.presidency.ucsb.edu/documents/republican-candidates-debate-north-charleston-south-carolina',
    'https://www.presidency.ucsb.edu/documents/republican-candidates-debate-las-vegas-nevada-0',
    'https://www.presidency.ucsb.edu/documents/republican-candidates-debate-milwaukee-wisconsin',
    'https://www.presidency.ucsb.edu/documents/republican-candidates-debate-boulder-colorado',
    'https://www.presidency.ucsb.edu/documents/republican-candidates-debate-simi-valley-california-0',
    'https://www.presidency.ucsb.edu/documents/republican-candidates-debate-cleveland-ohio'
]

In [22]:
url = "https://www.presidency.ucsb.edu/ws/index.php?pid=115148"
response = request.urlopen(url)

raw = response.read().decode('utf-8-sig')
raw

'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML+RDFa 1.0//EN"\n  "http://www.w3.org/MarkUp/DTD/xhtml-rdfa-1.dtd">\n<html lang="en" dir="ltr" prefix="content: http://purl.org/rss/1.0/modules/content/ dc: http://purl.org/dc/terms/ foaf: http://xmlns.com/foaf/0.1/ og: http://ogp.me/ns# rdfs: http://www.w3.org/2000/01/rdf-schema# sioc: http://rdfs.org/sioc/ns# sioct: http://rdfs.org/sioc/types# skos: http://www.w3.org/2004/02/skos/core# xsd: http://www.w3.org/2001/XMLSchema#">\n<head profile="http://www.w3.org/1999/xhtml/vocab">\n  <meta charset="utf-8"><script type="text/javascript">(window.NREUM||(NREUM={})).init={ajax:{deny_list:["bam.nr-data.net"]}};(window.NREUM||(NREUM={})).loader_config={licenseKey:"dee899de70",applicationID:"80106271"};;/*! For license information please see nr-loader-rum-1.242.0.min.js.LICENSE.txt */\n(()=>{"use strict";var e,t,n={234:(e,t,n)=>{n.d(t,{P_:()=>h,Mt:()=>m,C5:()=>s,DL:()=>w,OP:()=>j,lF:()=>N,Yu:()=>_,Dg:()=>v,CX:()=>c,GE:()=>A,sU:()=>T});var r=n(8632),i=n(9

In [64]:
def text_cleaner(text):
    soup = BeautifulSoup(text, 'html.parser')
    [s.extract() for s in soup(['iframe', 'script'])]
    stripped_text = soup.get_text()
    stripped_text = re.sub('[\r\n|\r\n]+', '\n', stripped_text)
    stripped_text = re.sub(r'\d+', '', stripped_text)
    stripped_text = re.sub('’', '', stripped_text)
    #stripped_text = re.sub(r'[^A]')
    # iteratively add cleaning steps here
    return stripped_text

clean_text = text_cleaner(raw)

In [65]:
# Define a function to extract the desired portion of text
def extract_desired_text(text):
    # Use regular expressions to find the text between "MODERATORS" and "Filed Under"
    pattern = re.compile(r'MODERATORS(.*?)Filed Under', re.DOTALL)
    match = pattern.search(text)
    
    if match:
        desired_text = match.group(1).strip()
    else:
        desired_text = text
    
    return desired_text

In [41]:
# Initialize an empty list to store the responses
responses = []

# Iterate through the URL list
for url in dem_urls:
    try:
        # Send a request to the URL
        response = request.urlopen(url)

        # Read and decode the response
        raw = response.read().decode('utf-8-sig')

        # Clean the text using the text_cleaner function
        clean_text = text_cleaner(raw)

        # Extract the desired portion of text
        desired_text = extract_desired_text(clean_text)

        # Append the cleaned and desired text to the responses list
        responses.append(desired_text)
    except Exception as e:
        # Handle exceptions if the URL request fails
        print(f"Error fetching URL {url}: {e}")

# Now, 'responses' contains the cleaned and desired text for each URL in the same order as 'url_list'
for i, desired_text in enumerate(responses):
    print(f"Cleaned and Desired Response {i + 1}:\n{desired_text}\n")


Cleaned and Desired Response 1:
:Dana Bash (CNN);Ilia Calderón (Univision); andJake Tapper (CNN)
TAPPER: Good evening from Washington, D.C. And welcome to this unique event, the CNN-Univision Democratic presidential debate with the two leading candidates for the Democratic nomination, former Vice President Joe Biden and Vermont Senator Bernie Sanders.
Candidates, welcome, it's good to have you. I'm Jake Tapper, along with CNN's Dana Bash and Univision's Ilia Calderon.
BASH: We come together tonight at an extraordinary time in our country when people are worried about far more than just presidential politics. We're in a national emergency because of the devastating global pandemic of coronavirus. It has killed nearly , people around the world, and  dead and more than , known cases here in the United States. As a result, tonight's debate will focus heavily on the crisis.
CALDERON: The setting of this debate is also different. To reduce unnecessary risk of transmission of the virus, CNN, 

In [66]:
import urllib.request as request
from bs4 import BeautifulSoup
import re
import pandas as pd

# Define the text cleaner function
def text_cleaner(text):
    soup = BeautifulSoup(text, 'html.parser')
    [s.extract() for s in soup(['iframe', 'script'])]
    stripped_text = soup.get_text()
    stripped_text = re.sub('[\r\n|\r\n]+', '\n', stripped_text)
    stripped_text = re.sub(r'\d+', '', stripped_text)
    stripped_text = re.sub('’', '', stripped_text)
    # Add more cleaning steps here if needed
    return stripped_text

# Define a function to extract the desired portion of text
def extract_desired_text(text):
    # Use regular expressions to remove the line starting with 'MODERATOR' and everything between "MODERATORS" and "Filed Under"
    pattern = re.compile(r'MODERATOR.*?Filed Under', re.DOTALL)
    cleaned_text = pattern.sub('', text).strip()
    return cleaned_text

# Define a function to process a URL and return the cleaned and desired text
def process_url(url):
    try:
        # Send a request to the URL
        response = request.urlopen(url)

        # Read and decode the response
        raw = response.read().decode('utf-8-sig')

        # Clean the text using the text_cleaner function
        clean_text = text_cleaner(raw)

        # Extract the desired portion of text using the extract_desired_text function
        desired_text = extract_desired_text(clean_text)

        return desired_text
    except Exception as e:
        # Handle exceptions if the URL request fails
        print(f"Error fetching URL {url}: {e}")
        return None

# Initialize an empty list to store the responses
responses = []

# Iterate through the URL list and process each URL
for url in dem_urls:
    desired_text = process_url(url)
    if desired_text is not None:
        responses.append(desired_text)

# Create a DataFrame with the responses
df = pd.DataFrame({'Text': responses})

# Now, 'df' is a DataFrame where each row corresponds to a cleaned and desired text from a URL
print(df)

                                                 Text
0   Democratic Candidates Debate in Washington, DC...
1   Democratic Candidates Debate in Charleston, So...
2   Democratic Candidates Debate in Las Vegas, Nev...
3   Democratic Candidates Debate in Manchester, Ne...
4   Democratic Candidates Debate in Des Moines, Io...
5   Democratic Candidates Debate in Los Angeles, C...
6   Democratic Candidates Debate in Atlanta, Georg...
7   Democratic Candidates Debate in Westerville, O...
8   Democratic Candidates Debate in Houston, Texas...
9   Democratic Candidates Debate in Detroit, Michi...
10  Democratic Candidates Debate in Detroit, Michi...
11  Democratic Candidates Debate in Miami, Florida...
12  Democratic Candidates Debate in Miami, Florida...


In [67]:
import urllib.request as request
from bs4 import BeautifulSoup
import re
import pandas as pd

# Define the text cleaner function
def text_cleaner(text):
    soup = BeautifulSoup(text, 'html.parser')
    [s.extract() for s in soup(['iframe', 'script'])]
    stripped_text = soup.get_text()
    stripped_text = re.sub('[\r\n|\r\n]+', '\n', stripped_text)
    stripped_text = re.sub(r'\d+', '', stripped_text)
    stripped_text = re.sub('’', '', stripped_text)
    # Add more cleaning steps here if needed
    return stripped_text

# Define a function to extract the desired portion of text
def extract_desired_text(text):
    # Use regular expressions to find the text between "MODERATORS" and "Filed Under"
    pattern = re.compile(r'MODERATORS(.*?)Filed Under', re.DOTALL)
    match = pattern.search(text)
    
    if match:
        desired_text = match.group(1).strip()
    else:
        desired_text = text
    
    return desired_text

# Define a function to process a URL and return the cleaned and desired text as a list of lines
def process_url(url):
    try:
        # Send a request to the URL
        response = request.urlopen(url)

        # Read and decode the response
        raw = response.read().decode('utf-8-sig')

        # Clean the text using the text_cleaner function
        clean_text = text_cleaner(raw)

        # Extract the desired portion of text using the extract_desired_text function
        desired_text = extract_desired_text(clean_text)

        # Split the text into lines
        lines = desired_text.split('\n')

        return lines
    except Exception as e:
        # Handle exceptions if the URL request fails
        print(f"Error fetching URL {url}: {e}")
        return None

# Initialize an empty list to store the responses
lines = []

# Iterate through the URL list and process each URL
for url in dem_urls:
    lines_from_url = process_url(url)
    if lines_from_url is not None:
        lines.extend(lines_from_url)

# Create a DataFrame with the lines
df = pd.DataFrame({'Text': lines})

# Now, 'df' is a DataFrame where each row corresponds to a line from the extracted text
# Add the democrat label to each row
df['Label'] = 'democrat'

In [68]:
print(df)

                                                   Text     Label
0     :Dana Bash (CNN);Ilia Calderón (Univision); an...  democrat
1     TAPPER: Good evening from Washington, D.C. And...  democrat
2     Candidates, welcome, it's good to have you. I'...  democrat
3     BASH: We come together tonight at an extraordi...  democrat
4     CALDERON: The setting of this debate is also d...  democrat
...                                                 ...       ...
9025  WARREN: Thank you. It's a great honor to be he...  democrat
9026  GUTHRIE: Senator We would like to thank all of...  democrat
9027  HOLT: We certainly hope you will join us then ...  democrat
9028  NOTE: Because of the large number of candidate...  democrat
9029  Presidential Candidate Debates, Democratic Can...  democrat

[9030 rows x 2 columns]


In [69]:
import urllib.request as request
from bs4 import BeautifulSoup
import re
import pandas as pd

# Define the text cleaner function
def text_cleaner(text):
    soup = BeautifulSoup(text, 'html.parser')
    [s.extract() for s in soup(['iframe', 'script'])]
    stripped_text = soup.get_text()
    stripped_text = re.sub('[\r\n|\r\n]+', '\n', stripped_text)
    stripped_text = re.sub(r'\d+', '', stripped_text)
    stripped_text = re.sub('’', '', stripped_text)
    # Add more cleaning steps here if needed
    return stripped_text

# Define a function to extract the desired portion of text
def extract_desired_text(text):
    # Use regular expressions to find the text between "MODERATORS" and "Filed Under"
    pattern = re.compile(r'MODERATORS(.*?)Filed Under', re.DOTALL)
    match = pattern.search(text)
    
    if match:
        desired_text = match.group(1).strip()
    else:
        desired_text = text
    
    return desired_text

# Define a function to process a URL and return the cleaned and desired text as a list of lines
def process_url(url):
    try:
        # Send a request to the URL
        response = request.urlopen(url)

        # Read and decode the response
        raw = response.read().decode('utf-8-sig')

        # Clean the text using the text_cleaner function
        clean_text = text_cleaner(raw)

        # Extract the desired portion of text using the extract_desired_text function
        desired_text = extract_desired_text(clean_text)

        # Split the text into lines
        linesr = desired_text.split('\n')

        return lines
    except Exception as e:
        # Handle exceptions if the URL request fails
        print(f"Error fetching URL {url}: {e}")
        return None

# Initialize an empty list to store the responses
linesr = []

# Iterate through the URL list and process each URL
for url in rep_urls:
    lines_from_url = process_url(url)
    if lines_from_url is not None:
        linesr.extend(lines_from_url)

# Create a DataFrame with the lines
dfr = pd.DataFrame({'Text': linesr})

# Now, 'df' is a DataFrame where each row corresponds to a line from the extracted text
# Add the democrat label to each row
dfr['Label'] = 'republican'

In [70]:
dfr

,Text,Label
0,:Dana Bash (CNN);Ilia Calderón (Univision); an...,republican
1,"TAPPER: Good evening from Washington, D.C. And...",republican
2,"Candidates, welcome, it's good to have you. I'...",republican
3,BASH: We come together tonight at an extraordi...,republican
4,CALDERON: The setting of this debate is also d...,republican
...,...,...
108355,WARREN: Thank you. It's a great honor to be he...,republican
108356,GUTHRIE: Senator We would like to thank all of...,republican
108357,HOLT: We certainly hope you will join us then ...,republican
108358,NOTE: Because of the large number of candidate...,republican


In [71]:
combined_df = pd.concat([df, dfr], ignore_index=True)

# The combined_df now contains all rows from both df and dfr
print(combined_df)

                                                     Text       Label
0       :Dana Bash (CNN);Ilia Calderón (Univision); an...    democrat
1       TAPPER: Good evening from Washington, D.C. And...    democrat
2       Candidates, welcome, it's good to have you. I'...    democrat
3       BASH: We come together tonight at an extraordi...    democrat
4       CALDERON: The setting of this debate is also d...    democrat
...                                                   ...         ...
117385  WARREN: Thank you. It's a great honor to be he...  republican
117386  GUTHRIE: Senator We would like to thank all of...  republican
117387  HOLT: We certainly hope you will join us then ...  republican
117388  NOTE: Because of the large number of candidate...  republican
117389  Presidential Candidate Debates, Democratic Can...  republican

[117390 rows x 2 columns]


In [72]:
# print value counts for each label
print(combined_df['Label'].value_counts())

republican    108360
democrat        9030
Name: Label, dtype: int64
